In [1]:
import cobra
from cobra.io import load_model, read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import scipy
from scipy.stats import pearsonr
import numpy as np
import matplotlib.pyplot as plt
import importlib
import gc
from cobra.sampling import sample
from matplotlib.ticker import MaxNLocator

In [7]:
organism="Ecoli_iJO1366"
BOUND = 30000

In [15]:
model = load_model("iJO1366")
# Actualizar la regla de reacción de cada gen para cada reacción con el ID de la reacción
for reaction in model.reactions:
    reaction.gene_reaction_rule = reaction.id

In [6]:
# Crear archivo de medio por defecto  
solution=model.optimize()
with open('medium.csv', 'w', newline="\n") as f:
    print("Metabolite_Name", "Reaction_ID", sep="\t", file=f)
    for reaction in model.summary().uptake_flux.index:
        metabolite = reaction.split("_")[1]
        print(metabolite, reaction, sep="\t", file=f)

In [7]:
for reaction in model.reactions:
    if reaction.lower_bound <0: reaction.lower_bound=-BOUND
    if reaction.lower_bound >0: reaction.lower_bound=0
    if reaction.upper_bound <0: reaction.upper_bound=0
    if reaction.upper_bound >0: reaction.upper_bound=BOUND

In [ ]:
# Do sampling
modelS = load_model("iJO1366")
for reaction in modelS.reactions:
    if reaction.lower_bound <0: reaction.lower_bound=-BOUND
    if reaction.lower_bound >0: reaction.lower_bound=0
    if reaction.upper_bound <0: reaction.upper_bound=0
    if reaction.upper_bound >0: reaction.upper_bound=BOUND
for reaction in modelS.reactions:
    LB, UB = reaction.bounds
    if LB==-1000: LB = -BOUND
    if UB== 1000: UB =  BOUND
    reaction.bounds=(LB,UB)
modelS.reactions.EX_glc__D_e.lower_bound = -10
modelS.reactions.EX_cbl1_e.bounds = (-BOUND,0)
s = sample(modelS, 1000)
s.to_csv('sampled_flux_distributions.csv') 
s = pd.read_csv('sampled_flux_distributions.csv',index_col=0)

In [8]:
cobra.io.write_sbml_model(model, "ecoli_iJO1366.xml")

In [34]:
import pandas as pd
import os # 1. Import the 'os' module for directory and path operations

# 2. Define the names of the new directories
known_fluxes_dir = 'knownFluxes'
transcriptomes_dir = 'transcriptomes'
inputFile_dir = "inputFiles"

# Create the directories before the loop starts.
# 'exist_ok=True' prevents an error if the directories already exist.
os.makedirs(known_fluxes_dir, exist_ok=True)
os.makedirs(transcriptomes_dir, exist_ok=True)
os.makedirs(inputFile_dir, exist_ok=True)
print(f"Ensured directories '{known_fluxes_dir}' and '{transcriptomes_dir}' exist.")


# Now, start the loop to process each row and append to the file.
for i in range(s.shape[0]):
    
    fluxesRef = s.loc[i]
    expressions = fluxesRef.abs()

    # --- Create iteration-specific files inside their respective folders ---
    
    # 3. Use os.path.join() to create the full path for each file.
    # This correctly adds the folder name to the path.
    known_fluxes_filename = os.path.join(known_fluxes_dir, f'knownFluxes_{i}.csv')
    transcriptome_filename = os.path.join(transcriptomes_dir, f'transcriptome_{i}.csv')
    inputFile_filename     = os.path.join(inputFile_dir, f"inputData_{i}.csv")

    # Create the knownFluxes file for the current iteration
    with open(known_fluxes_filename, 'w', newline="\n") as f:
        print("Reaction_ID", "Metabolite_ID", "Reaction_Flux", sep="\t", file=f)
        intakeRxns = ["EX_glc__D_e"]
        for Reaction_ID in intakeRxns:
            Metabolite_ID = Reaction_ID[3::]
            Reaction_Flux = fluxesRef[Reaction_ID]
            print(Reaction_ID, Metabolite_ID, Reaction_Flux, sep="\t", file=f)
    
    # Create the transcriptome file for the current iteration
    with open(transcriptome_filename, 'w', newline="\n") as f:
        print("Gene_ID", "Expression", sep="\t", file=f)
        for reactionID in expressions.index:
            print(reactionID, expressions[reactionID], sep="\t", file=f)
    
    # Open the inputData.csv file 
    with open(inputFile_filename, 'w', newline="\n") as input_data_file:
        # Write the header to the main file just one time.
        print("Organism", "Condition", "GeneExpFile", "Medium", "Network", "KnownFluxes", sep="\t", file=input_data_file)    
        # The variables now hold the full paths (e.g., "knownFluxes/knownFluxes_0.csv")
        # so this part works perfectly without further changes.
        Organism = organism
        Condition = f"glucose_iter_{i}"
        GeneExpFile = transcriptome_filename
        Medium = "medium.csv"
        Network = "ecoli_iJO1366.xml"
        KnownFluxes = known_fluxes_filename
        # Print the new line to the already-open input_data_file
        print(Organism, Condition, GeneExpFile, Medium, Network, KnownFluxes, sep="\t", file=input_data_file)

Ensured directories 'knownFluxes' and 'transcriptomes' exist.
